In [30]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import tensorflow as tf

In [31]:
TRAIN = [line.rstrip() for line in open("data/modelnet40_ply_hdf5_2048/train_files.txt")]
TEST = [line.rstrip() for line in open("data/modelnet40_ply_hdf5_2048/test_files.txt")]
LABEL_NAMES = [line.rstrip() for line in open("data/modelnet40_ply_hdf5_2048/shape_names.txt")]

In [32]:
def load_h5(h5_filename):
    f = h5py.File(h5_filename)
    data = f['data'][:]
    label = f['label'][:]
    return (data, label)

In [33]:
def visualize3D(x, y, z):
    fig = plt.figure()
    ax = Axes3D(fig)
    ax.scatter(x, y, z)
    plt.show()

In [34]:
def pointCloudTransformation(pointCloud, angle, translation):
    transformMatrixZ = np.array([
                       [np.cos(angle), -np.sin(angle), 0],
                       [np.sin(angle), np.cos(angle) , 0],
                       [0            , 0             , 1]
                      ])
    
    transformMatrixY = np.array([
                       [np.cos(angle) , 0, np.sin(angle)],
                       [0             , 1, 0            ],
                       [-np.sin(angle), 0, np.cos(angle)]
                      ])
    
    transformMatrixX = np.array([
                       [1, 0            ,              0],
                       [0, np.cos(angle), -np.sin(angle)],
                       [0, np.sin(angle),  np.cos(angle)]
                      ])
    pointCloud =  np.dot(pointCloud, transformMatrixX)
    pointCloud =  np.dot(pointCloud, transformMatrixY)
    pointCloud =  np.dot(pointCloud, transformMatrixZ)
    
    
#     transformMatrixZ = tf.constant([[tp.cos(angle), -tp.sin(angle), 0], [tp.sin(angle), tp.cos(angle) , 0], [0,0,1]])
#     transformMatrixY = tf.constant([[tp.cos(angle),0, tp.sin(angle)], [0,0,1], [-tp.sin(angle), 0, tp.cos(angle)]])
#     transformMatrixX = tf.constant([[1, 0,0], [0,tp.cos(angle), -tp.sin(angle)], [0, tp.sin(angle), tp.cos(angle)]])
#     pointCloud = tf.tensordot(pointCloud, transformMatrixX, 1)
#     pointCloud = tf.tensordot(pointCloud, transformMatrixY, 1)
#     pointCloud = tf.tensordot(pointCloud, transformMatrixZ, 1)
    return pointCloud

In [71]:
def generteRandomPoints(noOfPoints, radius, center):
    x = np.empty(noOfPoints)
    y = np.empty(noOfPoints)
    z = np.empty(noOfPoints)
    
    for i in range(noOfPoints):
        phi = np.random.uniform(0,2*np.pi)
        theta = np.random.uniform(0,np.pi)
        randomRadius = radius * (np.random.uniform(0, 1))
        x[i] = randomRadius * np.sin(theta) * np.cos(phi)
        y[i] = randomRadius * np.sin(theta) * np.sin(phi)
        z[i] = randomRadius * np.cos(theta)
    x += center[0]
    y += center[1]
    z += center[2]
    return x, y, z

#     radius = tf.constant([radius])
#     phi = tf.random_normal([0,2*np.pi])
#     theta = tf.random_normal([0,np.pi])
#     randomRadius = radius * (tf.random_normal([0, 1]))
#     xCoordinate = radius * tf.sin(theta) * tf.cos(phi) + center
#     yCoordinate = radius * tf.sin(theta) * tf.sin(phi)
#     zCoordinate = radius * tf.cos(theta)

In [ ]:
def Reconstruction(pointCloud, noOfPoints):
    reconstructedPoints = []
    pointPerSample = noOfPoints/int(pointCloud.shape[1])
    for point in pointCloud:
        point = reshape(point, [-1])
        reconstructedPoints.append(generateRandomPoints(pointPerSample, .05, point ))
    return np.array(reconstructedPoints)

In [68]:
models = []
labels = []
for file in TRAIN:
    current_data, current_label = load_h5(file)
    for index in range(len(current_data)):
        temp = pointCloudTransformation(current_data[index], np.pi/2, 0)    
        models.append(temp)
        labels.append(current_label[index])
models = np.array(models)
labels = np.array(models)

In [70]:
tf.reset_default_graph()
BATCH_SIZE = 100
x = tf.placeholder(tf.float32, shape=[BATCH_SIZE, 3*2048])
layer1 = tf.contrib.layers.fully_connected(x, 3072,activation_fn=tf.nn.relu)
layer2 = tf.contrib.layers.fully_connected(layer1, 1536,activation_fn=tf.nn.relu)
layer3 = tf.contrib.layers.fully_connected(layer2, 768,activation_fn=tf.nn.relu)
output = tf.contrib.layers.fully_connected(layer3, 384,activation_fn=tf.nn.relu)
reconstructedPoints = Reconstruction(output, 3*2048)
loss = tf.reduce_mean(tf.losses.mean_squared_error(output,reconstructedPoints))  
# train_op = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)
init = tf.global_variables_initializer()

TypeError: 'Tensor' object is not iterable.

In [ ]:
#Read Data plot 3d points
#Compress data code
#Reconstruct by making sphere and putting random points in that 
